In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [2]:
df3 = pd.read_parquet(data_path + 'data/train00.parquet')
# df3 = df3.drop('Unnamed: 0', axis=1, inplace=True)
# df3.to_parquet(data_path + 'data/train0.parquet')
# df3.info()
# df3 = df3.loc[:400000-1,:]
df3

,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회경과개월수_신용,...,연체입금원금_B0M,정상청구원금_B5M,선입금원금_B5M,정상입금원금_B5M,연체입금원금_B5M,연속무실적개월수_기본_24M_카드,연속유실적개월수_기본_24M_카드,신청건수_ATM_CA_B0,신청건수_ATM_CA_R6M,이용금액대
0,1,2,D,1,1,0,1,1,1,67,...,8104,14958,0,9205,5752,0,13,0,11,10000
1,0,1,E,1,1,1,1,1,1,12,...,826,3367,0,2546,821,0,12,0,0,3000
2,0,1,C,1,1,0,1,1,1,124,...,9364,23963,0,16949,7014,0,8,0,0,10000
3,1,2,D,1,1,0,1,2,2,27,...,10923,19614,0,8418,11196,0,5,0,0,10000
4,1,2,E,1,1,1,1,1,1,2,...,0,0,0,0,0,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1,5,E,1,1,1,1,1,1,205,...,0,579,0,0,579,12,0,0,0,0
399996,1,3,D,1,1,1,1,1,1,13,...,1910,30348,0,30348,0,0,20,0,0,10000
399997,0,1,C,1,1,0,1,1,1,111,...,856,11489,0,5128,6360,0,24,0,0,5000
399998,0,2,E,1,1,1,1,1,1,67,...,507,672,0,0,672,14,0,0,0,0


In [3]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(sampling_strategy='not majority')

In [ ]:
df3.to_parquet(data_path + 'data/train00.parquet', index=False)

In [4]:
# df3 = df3.loc[:400000,:]
x = df3.drop(['Segment'], axis=1,)
# '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M'는 encoding 문제로 제거
y = df3['Segment']

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x= scaler.fit_transform(x)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42, stratify=y
)

In [7]:
train_x,train_y=smote.fit_resample(X_train,y_train)
train_y.value_counts()

,count
Segment,
D,288308
E,288308
C,288308
A,288308
B,288308


In [8]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 23.4 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(train_y)
y_test_enc = le.transform(y_test)

In [9]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    depth= 10,
    loss_function="MultiClass",
    random_state=42,
)

model.fit(train_x, train_y)
y_pred = model.predict(X_test)


Learning rate set to 0.114663
0:	learn: 1.3391946	total: 14.2s	remaining: 3h 56m 33s
1:	learn: 1.1700582	total: 28.3s	remaining: 3h 55m 5s
2:	learn: 1.0424943	total: 42.4s	remaining: 3h 54m 41s
3:	learn: 0.9432496	total: 56.8s	remaining: 3h 55m 55s
4:	learn: 0.8638894	total: 1m 11s	remaining: 3h 55m 36s
5:	learn: 0.7959792	total: 1m 25s	remaining: 3h 55m 40s
6:	learn: 0.7401864	total: 1m 39s	remaining: 3h 54m 53s
7:	learn: 0.6915760	total: 1m 54s	remaining: 3h 57m 19s
8:	learn: 0.6503543	total: 2m 8s	remaining: 3h 56m 34s
9:	learn: 0.6121668	total: 2m 23s	remaining: 3h 57m 1s
10:	learn: 0.5802542	total: 2m 38s	remaining: 3h 56m 46s
11:	learn: 0.5529411	total: 2m 52s	remaining: 3h 57m 8s
12:	learn: 0.5277342	total: 3m 6s	remaining: 3h 56m 36s
13:	learn: 0.5057992	total: 3m 21s	remaining: 3h 56m 14s
14:	learn: 0.4857223	total: 3m 35s	remaining: 3h 55m 47s
15:	learn: 0.4689109	total: 3m 49s	remaining: 3h 54m 57s
16:	learn: 0.4536810	total: 4m 3s	remaining: 3h 54m 18s
17:	learn: 0.4386616	

In [10]:
from sklearn.metrics import f1_score
y_pred = model.predict(X_test)
# f1-micro 평가
f1_micro = f1_score(y_test, y_pred, average='micro')
print(f"F1 score (micro): {f1_micro:.4f}")

F1 score (micro): 0.9117


In [11]:
# f1-micro 평가
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"F1 score (macro): {f1_macro:.4f}")

F1 score (macro): 0.5280


In [ ]:
y_pred

array([['D'],
       ['E'],
       ['E'],
       ...,
       ['E'],
       ['E'],
       ['E']], dtype=object)

In [12]:
import pickle
with open(data_path + 'data/03_model.dat', 'wb') as fp:
    pickle.dump(model, fp)

In [14]:
yp = pd.DataFrame(y_pred)

In [15]:
yp.value_counts()

,count
0,
E,32279
D,5585
C,2125
A,10
B,1
